In [1]:
import os
import pickle

import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, GAE, Node2Vec, SAGEConv
from torch_geometric.utils import to_undirected, negative_sampling
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.loader import NeighborLoader

from decimal import Decimal, getcontext
from datetime import datetime, timezone
from collections import defaultdict, Counter
from joblib import dump, load

In [3]:
raw_train_df = pd.read_csv('train (1).csv')
raw_train_df

,id,target,767,766,765,764,763,762,761,760,...,9,8,7,6,5,4,3,2,1,0
0,1,0,-0.027923,0.044652,0.024014,-0.009848,-0.261522,0.256826,-0.050686,-0.086738,...,0.787836,-0.288956,0.556196,-0.580039,-0.066076,-0.403333,-0.306235,0.129879,-0.305108,1.829782
1,2,0,-0.027923,0.044652,0.024014,-0.009848,-0.261522,0.256826,-0.050686,-0.086738,...,0.787836,-0.288956,0.556196,-0.580039,-0.066076,-0.403333,-0.306235,0.129879,-0.305108,1.829782
2,4,0,-0.027923,0.044652,0.024014,-0.009848,-0.261522,0.256826,-0.050686,-0.086738,...,0.787836,-0.288956,0.556196,-0.580039,-0.066076,-0.403333,-0.306235,0.129879,-0.305108,1.829782
3,5,0,-0.027923,0.044652,0.024014,-0.009848,-0.261522,0.256826,-0.050686,-0.086738,...,0.787836,-0.288956,0.556196,-0.580039,-0.066076,-0.403333,-0.306235,0.129879,-0.305108,1.829782
4,7,0,-0.018281,0.037508,0.082157,0.120872,-0.061487,0.099580,-0.149192,-0.006453,...,0.966876,0.715060,-0.679073,0.007701,-0.744617,0.178890,0.012163,-0.626677,0.396385,2.143136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15829,19785,57,0.091960,-0.050702,0.060201,-0.062596,-0.096211,0.180081,-0.066306,0.049754,...,-0.430971,0.494280,-0.032868,-0.716817,-1.185531,-0.213318,-0.150297,-0.397840,-1.113600,1.442408
15830,19786,57,-0.019448,-0.059912,0.079663,-0.025937,-0.040015,0.145045,-0.041019,0.090989,...,-0.182306,0.530710,0.564926,-0.298283,-0.451402,0.118806,-0.070511,-0.219921,-0.592173,1.303567
15831,19787,23,0.038737,-0.098368,-0.180312,0.157929,-0.047813,0.187792,-0.213342,-0.137691,...,-0.108976,0.155940,-0.265013,-0.640379,0.729008,-0.213324,-0.322087,0.084669,0.207762,1.440589
15832,19790,52,-0.177773,0.028562,0.118652,-0.419880,-0.028193,-0.172422,-0.089091,0.059918,...,0.369025,-0.039625,-0.147634,0.514334,-0.588722,0.485571,-0.149175,0.829777,1.069023,2.214765


In [7]:
raw_train_df['target'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 15, 16, 17, 18,
       20, 21, 22, 25, 26, 28, 29, 30, 11, 33, 34, 35, 36, 37, 38, 19, 31,
       39, 24, 42, 43, 44, 45, 46, 47, 27, 48, 41, 49, 50, 51, 52, 54, 55,
       56, 57, 58, 59, 53, 60, 23, 40, 14, 61, 62, 63, 64, 65, 67, 32, 66,
       68, 69], dtype=int64)

In [4]:
raw_test_df = pd.read_csv('test (1).csv')
raw_test_df

,id,767,766,765,764,763,762,761,760,759,...,9,8,7,6,5,4,3,2,1,0
0,0,-0.113558,0.057501,-0.033186,-0.050327,0.009008,-0.159796,-0.202110,0.061212,0.094594,...,0.360014,0.544246,0.302459,0.620601,-0.083619,-0.770815,-0.422844,-0.013988,0.125635,1.705145
1,3,-0.063730,-0.137200,-0.022058,0.046249,-0.020714,-0.033404,-0.097084,0.015601,-0.054544,...,1.027847,-0.178545,0.082991,-0.693945,0.116560,-0.543442,0.906257,1.105021,0.289673,1.098968
2,6,0.105503,-0.032846,0.094267,-0.018496,-0.030531,-0.144394,0.008078,-0.042541,-0.016814,...,0.337400,0.059743,0.234613,-0.726217,-0.666962,-0.324834,-0.974187,-0.050589,0.399163,2.136273
3,17,-0.167171,-0.107339,-0.318118,0.110821,-0.089710,-0.085125,-0.197500,-0.200957,0.210670,...,0.327049,0.731411,0.535729,0.434602,-0.132322,0.149765,-0.383719,0.440560,-0.169332,1.074303
4,19,-0.013470,0.006680,-0.024772,-0.123437,-0.052524,-0.014114,-0.074786,-0.014438,0.024898,...,0.353890,-0.557812,-0.558715,-0.391345,-0.215380,-0.651793,-0.742601,0.326692,-0.355005,1.347859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3954,19781,0.047381,0.083490,-0.101291,-0.067637,-0.122052,0.140377,-0.067586,0.324571,0.114526,...,-0.198418,-0.019313,-0.019203,0.072593,-0.282034,0.259003,-0.322549,0.180135,-0.378451,1.311456
3955,19784,0.000670,-0.014312,0.050803,-0.020096,0.044747,-0.112488,0.017808,0.048268,0.129589,...,-0.403692,-0.252073,0.230208,-1.226392,0.411008,-0.427566,-0.196806,-0.251118,0.013319,1.147632
3956,19788,-0.033011,0.143096,0.220492,-0.033900,0.137237,0.227139,-0.137547,-0.012755,0.127434,...,-0.731517,0.760294,0.411503,0.328879,0.340879,0.369118,-0.192030,0.030741,-0.365289,1.277684
3957,19789,-0.046868,-0.015896,-0.131514,0.035267,-0.092612,0.173442,0.137644,-0.160942,-0.106897,...,-0.351143,0.253540,0.454129,-0.651563,0.374609,-0.778062,-0.228353,0.790958,0.176530,1.674738


In [8]:
raw_edge_index_df = pd.read_csv('edge_index (1).csv')
raw_edge_index_df

,Unnamed: 0,from,to
0,0,0,1227
1,1,0,4021
2,2,0,4056
3,3,0,4057
4,4,0,4105
...,...,...,...
126837,126837,19791,1485
126838,126838,19791,2099
126839,126839,19791,5100
126840,126840,19791,10850


array([    0,     1,     2, ..., 19790, 19791, 19792], dtype=int64)